In [1]:
import pandas as pd
import re
import operator
import string
import nltk 
from nltk.tag import pos_tag
pd.set_option('display.width', 10000)

In [2]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [17]:
copy = gg.copy(deep=True)

In [18]:
def removePunc(x):
#     new_string = x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\":()]+', '', x) #taking out punctuation i.e ? ! . ' : ( ) and " 
    x = re.sub(r'(RT|rt) ', '', x) #taking out the initial "RT "
    x= re.sub(r'(g|G)olden (g|G)lobes', '', x)
    return x.strip()
copy['text'] = copy['text'].str.lower()
copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))

In [45]:
test = list(copy['text'].values)
s = list(set(test))
df = pd.DataFrame(s, columns=['text'])

In [46]:
df = df.loc[df['text'].str.contains('presents')]
df.to_csv('test.csv', index=False)

In [38]:
def nGram(text):
    test = text
    firstsplit = test.split('present', 1)
    if len(firstsplit) > 1:
        left = firstsplit[0].split(' ')
        left.reverse()
        new_list = []
        x = ""
        for i in left:
            if(i == ""):
                new_list.append(x) #HACKS
                continue
            x =  i + " " + x 
            new_list.append(x)
        new_list = [x.rstrip() for x in new_list]
    else:
        new_list = []
    return new_list

def hashNgrams(dff):
    FINAL_NGRAMS = []
    NGRAMS_DICT = {}
    for i in range(len(dff)):
        l = nGram(dff['text'].iloc[i])
        for j in l:
            if j in NGRAMS_DICT:
                NGRAMS_DICT[j] += 1
            else:
                NGRAMS_DICT[j] = 1
    cd = sorted(NGRAMS_DICT.items(),key=lambda x: (x[1], len(x[0])),reverse=True)
    return cd

In [39]:
def findRows(x, contains, ncontains):
    s = "^"
    for word in contains:
        s += f'(?=.*{word})'
    for word in ncontains:
        s += f'(?!.*{word})'
    s += '.*$'
    if bool(re.match(s, x)):
        return True
    else:
        False
con = ['director', 'best']
noncon = []
df['Rows'] = df['text'].apply(lambda x: findRows(x, con, noncon))
print(len(df))
df.head(20)

ccc = df[~df['Rows'].isnull()]
print(len(ccc))
hashNgrams(ccc)

116942
519


[('', 4),
 ('ben affleck won two awards tonight - best director and best', 1),
 ('affleck won two awards tonight - best director and best', 1),
 ('won two awards tonight - best director and best', 1),
 ('two awards tonight - best director and best', 1),
 ('awards tonight - best director and best', 1),
 ('halle berry sexy as always at the', 1),
 ('tonight - best director and best', 1),
 ('berry sexy as always at the', 1),
 ('- best director and best', 1),
 ('best director and best', 1),
 ('sexy as always at the', 1),
 ('director and best', 1),
 ('as always at the', 1),
 ('yeah townies re', 1),
 ('always at the', 1),
 ('halle berry', 1),
 ('townies re', 1),
 ('and best', 1),
 ('at the', 1),
 ('berry', 1),
 ('best', 1),
 ('the', 1),
 ('re', 1)]

In [32]:
ccc.to_csv('test.csv', index=False)

In [ ]:
def findAllWinners(matrix):
    winners = []
    for i in range(0, len(matrix), 2):
        con = matrix[i]
        noncon = matrix[i+1]
        df['Rows'] = df['text'].apply(lambda x: findRows(x, con, noncon))
        ccc = df[~df['Rows'].isnull()]
        winner = hashNgrams(ccc)
        print(winner)
        winners.append(winner)
    return winners